Exercício 1

In [ ]:
import numpy as np
from scipy.linalg import solve_continuous_lyapunov

A = np.array([[0, 1],
              [-2, -3]])
Q = np.eye(2) 

# A'P + PA + Q = 0
P = solve_continuous_lyapunov(A.T, Q)

print("Matriz P:")
print(P)
print("\nTraço de P (Tr(P)):", np.trace(P))

In [ ]:
import cvxpy as cp
import numpy as np

A = np.array([[0, 1],
              [-2, -3]])
Q = np.eye(2) 

P = cp.Variable((2, 2), symmetric=True)

constraints = [
    P >> 0,  # P > 0 (definida positiva)
    A.T @ P + P @ A + Q << 0  # A'P + PA + Q < 0 (definida negativa)
]

objective = cp.Minimize(cp.trace(P))

problem = cp.Problem(objective, constraints)

problem.solve()

print("Matriz P (solução ótima):")
print(P.value)
print("\nTraço de P (Tr(P)):", np.trace(P.value))

Exercício 2

In [ ]:
import cvxpy as cp
import numpy as np

# System parameters
m1 = 1
m2 = 0.5
k1 = 1
k2 = 1
c0 = 2

A = np.array([
    [0, 0, 1, 0],
    [0, 0, 0, 1],
    [-((k1 + k2) / m1), k2 / m1, -c0 / m1, 0],
    [k2 / m2, -k2 / m2, 0, -c0 / m2]
])

B = np.array([
    [0],
    [0],
    [1 / m1],
    [0]
])

C = np.array([[0, 1, 0, 0]])

n_states = A.shape[0]

W = cp.Variable((n_states, n_states), symmetric=True)
rho = cp.Variable()

objective = cp.Minimize(rho)

constraints = [
    W >> 0, 
    rho >= cp.trace(C @ W @ C.T),
    A @ W + W @ A.T + B @ B.T << 0
]

problem = cp.Problem(objective, constraints)
problem.solve()

if problem.status == cp.OPTIMAL or problem.status == cp.OPTIMAL_INACCURATE:
    rho_star = rho.value
    h2_norm_squared = rho_star
    
    print(f'Status da solução: {problem.status}')
    print(f'Valor ótimo de \u03C1 (norma H2 ao quadrado): {h2_norm_squared:.4f}')
    print(f'Norma H2: {np.sqrt(h2_norm_squared):.4f}')

    print('Matriz W ótima:')
    print(W.value)
else:
    print(f'O problema não foi resolvido com sucesso. Status: {problem.status}')

Status da solução: optimal
Valor ótimo de ρ (norma H2 ao quadrado): 0.0926
Norma H2: 0.3043
Matriz W ótima:
[[ 1.57407421e-01  8.33333105e-02 -2.78350935e-08  3.70370627e-02]
 [ 8.33333105e-02  9.25925906e-02 -3.70370932e-02 -1.61823818e-08]
 [-2.78350935e-08 -3.70370932e-02  2.31481473e-01 -4.69721454e-09]
 [ 3.70370627e-02 -1.61823818e-08 -4.69721454e-09  1.85185400e-02]]


Exercício 3

In [ ]:
import cvxpy as cp
import numpy as np

m1 = 1
m2 = 0.5
k1 = 1
k2 = 1
c0 = 2

A = np.array([
    [0, 0, 1, 0],
    [0, 0, 0, 1],
    [-((k1 + k2) / m1), k2 / m1, -c0 / m1, 0],
    [k2 / m2, -k2 / m2, 0, -c0 / m2]
])

B = np.array([
    [0],
    [0],
    [1 / m1],
    [0]
])

C = np.array([[0, 1, 0, 0]])

D = np.zeros((1, 1))  # Zeros

n_states = A.shape[0]  # States number(4)
n_outputs = C.shape[0]  # Outputs number (1)

P = cp.Variable((n_states, n_states), symmetric=True)  
mu = cp.Variable() 

objective = cp.Minimize(mu)

block11 = A.T @ P + P @ A
block12 = P @ B
block13 = C.T
block21 = B.T @ P
block22 = -mu * np.eye(1)  
block23 = np.zeros((1, 1)) 
block31 = C
block32 = np.zeros((1, 1)) 
block33 = -np.eye(1) 

lmi_matrix = cp.bmat([
    [block11, block12, block13],
    [block21, block22, block23],
    [block31, block32, block33]
])

constraints = [
    P >> 0,
    lmi_matrix << 0 
]

problem = cp.Problem(objective, constraints)
problem.solve()

if problem.status == cp.OPTIMAL or problem.status == cp.OPTIMAL_INACCURATE:
    mu_star = mu.value
    h_inf_norm_squared = mu_star
    h_inf_norm = np.sqrt(mu_star)

    print(f'Status: {problem.status}')
    print(f'Valor ótimo (norma H infinito ao quadrado): {h_inf_norm_squared:.4f}')
    print(f'Norma H infinito: {h_inf_norm:.4f}')
    print('Matriz P ótima:')
    print(P.value)
else:
    print(f'Problema não resolvido. Status: {problem.status}')

Status da solução: optimal
Valor ótimo de μ (norma H∞ ao quadrado): 1.0000
Norma H∞: 1.0000


Exercício 4

In [ ]:
import cvxpy as cp
import numpy as np

# System dimensions
n = 4  # State dimension
mu = 1 # Control input T
mw = 1 # W
py = 3 # Output Y

k = 0.2  
f = 0.02  

A = np.array([
    [0, 0, 1, 0],
    [0, 0, 0, 1],
    [-k, k, -f, f],
    [k, -k, f, -f]
])

B1 = np.array([
    [0],
    [0],
    [0],
    [1]
])

B2 = np.array([
    [0],
    [0],
    [0],
    [1]
])

C = np.array([
    [1, 0, 0, 0],
    [0, 1, 0, 0],
    [0, 0, 0, 0]
])

D = np.array([
    [0],
    [0],
    [1]
])

rho = cp.Variable()

X = cp.Variable((py, py), symmetric=True)

Z = cp.Variable((mu, n))

W = cp.Variable((n, n), symmetric=True)

objective = cp.Minimize(rho)

constraints = [
    cp.bmat([
        [X, C @ W + D @ Z],
        [W @ C.T + Z.T @ D.T, W]
    ]) >> 0,
    rho >= cp.trace(X),

    cp.bmat([
        [A @ W + B2 @ Z + W @ A.T + Z.T @ B2.T, B1],
        [B1.T, -np.eye(mw)]
    ]) << 0, 
    W >> 0
]

problem = cp.Problem(objective, constraints)
problem.solve(solver='SCS')

if problem.status == cp.OPTIMAL or problem.status == cp.OPTIMAL_INACCURATE:
    print('\n--- Solução Ótima Encontrada ---')
    print(f'Status do problema: {problem.status}')

    h2_norm_squared = rho.value
    print(f"Custo H2 ao quadrado (ρ*): {h2_norm_squared:.4f}")
    print(f"Norma H2 (√ρ*): {np.sqrt(h2_norm_squared):.4f}")

    # Feedback states gain K = ZW⁻¹
    K = Z.value @ np.linalg.inv(W.value)
    print(f'Matriz de ganho de realimentação de estados K: {K}')

    A_cl = A + B2 @ K
    eigvals_cl = np.linalg.eigvals(A_cl)
    print(f'Autovalores do sistema em malha fechada (parte real): {np.real(eigvals_cl)}')
    if np.max(np.real(eigvals_cl)) < 0:
        print('O sistema em malha fechada é assintoticamente estável.')
    else:
        print('O sistema em malha fechada NÃO é assintoticamente estável.')

else:
    print(f'\nO problema não convergiu para uma solução ótima. Status: {problem.status}')


--- Solução Ótima Encontrada ---
Status do problema: optimal
Custo H2 ao quadrado (ρ*): 1.6289
Norma H2 (√ρ*): 1.2763
Matriz de ganho de realimentação de estados K: [[-0.09804839 -1.31626989 -2.14973271 -1.62908004]]
Autovalores do sistema em malha fechada (parte real): [-0.21984528 -0.21984528 -0.61469474 -0.61469474]
O sistema em malha fechada é assintoticamente estável.


Exercício 5

In [ ]:
import cvxpy as cp
import numpy as np

# --- 1. Definir as dimensões do sistema ---
n = 4  # Dimensão do estado (theta1, theta2, dtheta1, dtheta2)
mu = 1 # Dimensão da entrada de controle (T)
mw = 1 # Dimensão da perturbação (w)
py = 3 # Dimensão da saída (y)

# --- 2. Definir os parâmetros do modelo do satélite ---
k = 0.2   # Constante da mola (valor nominal)
f = 0.02  # Amortecimento viscoso (valor nominal)

# --- 3. Definir as matrizes do sistema ---
A = np.array([
    [0, 0, 1, 0],
    [0, 0, 0, 1],
    [-k, k, -f, f],
    [k, -k, f, -f]
])

B1 = np.array([
    [0],
    [0],
    [0],
    [1]
])

B2 = np.array([
    [0],
    [0],
    [0],
    [1]
])

C = np.array([
    [1, 0, 0, 0],
    [0, 1, 0, 0],
    [0, 0, 0, 0]
])

D = np.array([
    [0],
    [0],
    [1]
])

# --- 4. Declarar as variáveis de otimização CVXPY ---
# mu: escalar a ser minimizado (quadrado da norma H_infinito)
mu = cp.Variable()

# Z: Matriz (mu x n)
Z = cp.Variable((mu, n))

# W: Matriz simétrica definida positiva (n x n)
W = cp.Variable((n, n), symmetric=True)

# --- 5. Definir a função objetivo ---
objective = cp.Minimize(mu)

# --- 6. Definir as restrições LMI ---
constraints = [
    # Restrição 1: LMI para o ganho H_infinito
    cp.bmat([
        [A @ W + W @ A.T + B2 @ Z + Z.T @ B2.T, W @ C.T + Z.T @ D.T, B1],
        [C @ W + D @ Z, -np.eye(py), np.zeros((py, mw))],
        [B1.T, np.zeros((mw, py)), -mu * np.eye(mw)]
    ]) << 0,  # Matriz definida negativa

    # Restrição 2: W deve ser definida positiva
    W >> 0,

    # Restrição 3: mu deve ser não-negativo
    mu >= 0
]

# --- 7. Criar e resolver o problema CVXPY ---
problem = cp.Problem(objective, constraints)
problem.solve(solver='SCS')

# --- 8. Exibir os resultados ---
if problem.status == cp.OPTIMAL or problem.status == cp.OPTIMAL_INACCURATE:
    print("\n--- Solução Ótima Encontrada ---")
    print(f"Status do problema: {problem.status}")

    # Valor do custo H_infinito ao quadrado
    h_inf_norm_squared = mu.value
    print(f"Custo H_infinito ao quadrado (µ*): {h_inf_norm_squared:.4f}")
    
    # Norma H_infinito
    if h_inf_norm_squared >= 0:
        h_inf_norm = np.sqrt(h_inf_norm_squared)
        print(f"Norma H_infinito (√µ*): {h_inf_norm:.4f}")
    else:
        print("Erro: O valor de mu é negativo, a norma H_infinito não pode ser calculada.")

    # Ganho de realimentação de estados K = ZW⁻¹
    K = Z.value @ np.linalg.inv(W.value)
    print("\nMatriz de ganho de realimentação de estados K:")
    print(K)

    # Verificar estabilidade do sistema em malha fechada
    A_cl = A + B2 @ K
    eigvals_cl = np.linalg.eigvals(A_cl)
    print("\nAutovalores do sistema em malha fechada (parte real):")
    print(np.real(eigvals_cl))
    if np.max(np.real(eigvals_cl)) < 0:
        print("O sistema em malha fechada é assintoticamente estável.")
    else:
        print("O sistema em malha fechada NÃO é assintoticamente estável.")

else:
    print(f"\nO problema não convergiu para uma solução ótima. Status: {problem.status}")

Exercício 6

In [ ]:
import cvxpy as cp
import numpy as np

# --- 1. Definir as dimensões do sistema ---
n = 4  # Dimensão do estado (theta1, theta2, dtheta1, dtheta2)
m_u = 1 # Dimensão da entrada de controle (T)
m_w = 1 # Dimensão da perturbação (w)
p_y = 3 # Dimensão da saída (y)

# --- 2. Definir os intervalos dos parâmetros incertos ---
k_min, k_max = 0.09, 0.4
f_min, f_max = 0.0038, 0.04

# --- 3. Definir as matrizes do sistema base (constantes para todos os vértices) ---
B1 = np.array([[0], [0], [0], [1]])
B2 = np.array([[0], [0], [0], [1]])
C = np.array([
    [1, 0, 0, 0],
    [0, 1, 0, 0],
    [0, 0, 0, 0]
])
D = np.array([[0], [0], [1]])

# --- 4. Gerar as matrizes A para cada vértice do politopo ---
k_vals = [k_min, k_max]
f_vals = [f_min, f_max]

A_vertices = []
vertex_params = []  # Para armazenar os pares (k, f)

for k_val in k_vals:
    for f_val in f_vals:
        A_vertex = np.array([
            [0, 0, 1, 0],
            [0, 0, 0, 1],
            [-k_val, k_val, -f_val, f_val],
            [k_val, -k_val, f_val, -f_val]
        ])
        A_vertices.append(A_vertex)
        vertex_params.append({'k': k_val, 'f': f_val})

N = len(A_vertices)  # Número de vértices (N=4)

# --- 5. Declarar as variáveis de otimização CVXPY ---
rho = cp.Variable(pos=True)  # Escalar a ser minimizado (custo H2 garantido)
X = cp.Variable((n, n), symmetric=True)  # Matriz auxiliar (n x n)
Z = cp.Variable((m_u, n))  # Z = K * W (m_u x n)
W = cp.Variable((n, n), symmetric=True)  # W = P^-1 (n x n)

# --- 6. Definir a função objetivo ---
objective = cp.Minimize(rho)

# --- 7. Definir as restrições LMI ---
constraints = [
    W >> 0,  # W deve ser definida positiva
    rho >= cp.trace(X),  # Restrição para o custo H2 garantido
    rho >= 0  # Rho deve ser não negativo
]

for i in range(N):
    Ai = A_vertices[i]
    B1i, B2i, Ci, Di = B1, B2, C, D

    # Restrição 1: [ X B1i' ; B1i W ] > 0
    constraints.append(
        cp.bmat([
            [X, B1i.T],
            [B1i, W]
        ]) >> 0
    )

    # Restrição 2: [ AiW + B2iZ + WAi' + Z'B2i' WCi' + Z'Di' ; CiW + DiZ -I ] < 0
    constraints.append(
        cp.bmat([
            [Ai @ W + B2i @ Z + W @ Ai.T + Z.T @ B2i.T, W @ Ci.T + Z.T @ Di.T],
            [Ci @ W + Di @ Z, -np.eye(p_y)]
        ]) << 0
    )

# --- 8. Criar e resolver o problema CVXPY ---
problem = cp.Problem(objective, constraints)
problem.solve(solver='SCS', verbose=True)

# --- 9. Exibir os resultados ---
if problem.status == cp.OPTIMAL or problem.status == cp.OPTIMAL_INACCURATE:
    print("\n--- Solução Ótima Encontrada ---")
    print(f"Status do problema: {problem.status}")

    guaranteed_h2_cost_squared = rho.value
    print(f"Custo H2 garantido ao quadrado (ρ*): {guaranteed_h2_cost_squared:.4f}")

    if guaranteed_h2_cost_squared >= 0:
        guaranteed_h2_norm = np.sqrt(guaranteed_h2_cost_squared)
        print(f"Norma H2 garantida (√ρ*): {guaranteed_h2_norm:.4f}")
    else:
        print("Erro: O valor de rho é negativo, a norma H2 não pode ser calculada.")

    # Calcular o ganho de realimentação de estados K = ZW⁻¹
    K = Z.value @ np.linalg.inv(W.value)
    print("\nMatriz de ganho de realimentação de estados K:")
    print(K)

else:
    print(f"\nO problema não convergiu para uma solução ótima. Status: {problem.status}")